In [1]:
!pip install transformers==4.44.0
!pip install accelerate==1.2.1
!pip install python-dotenv==1.0.1
!pip install requests==2.31.0
!pip install PyYAML==6.0
!pip install Jinja2==3.1.0
!pip install tornado==6.2
!pip install outlines==0.1.12
!pip install datasets

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires requests>=2.32.2, but you have requests 2.31.0 which is incompatible.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0


In [2]:
import os
import requests          # For making API calls
import json              # For parsing and working with JSON data
import datetime          # For handling dates and validating input
from IPython import get_ipython
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import ipywidgets as widgets
from dotenv import load_dotenv
import outlines
import datasets
import torch


In [ ]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def search_flights(Origin, destination, start_date, end_date):
    """
    Queries Flight Scaper's roundtrip flight search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/flights/search-roundtrip"
    headers = {
      "x-rapidapi-key": os.getenv("API_KEY"),
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {
        "origin": Origin,  
        "destination": destination,
        "departureDate": start_date,
        "returnDate": end_date,
        "adults": 1
    }

    response = requests.get(url, headers=headers, params=params)
    return response.json()

def search_hotels(location, check_in, check_out):
    """
    Queries Flight Scaper's hotel search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/hotels/search"
    headers = {
      "x-rapidapi-key": os.getenv("API_KEY"),
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {
        "location": location,
        "checkIn": check_in,
        "checkOut": check_out,
        "adults": 1
    }

    response = requests.get(url, headers=headers, params=params)
    return response.json()

class LlamaTravelBot:
    def __init__(self):
        self.travel_details = {}  # Store travel details
        self.tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.model = AutoModelForCausalLM.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.generator = pipeline(task="text-generation", model=self.model, tokenizer=self.tokenizer, device=0)
        self.chat_history= []
        self.details={}

    def update_travel_details(self):
         """
        Update the travel details dictionary with new information.
         """
         self.travel_details.update(self.details)

    

    def llama_ask(self, prompt):
        """
        Generate a response from the Llama model.
        """
        self.chat_history=[
            {"role": "system",
        "content": " You are an intelligent and conversational travel assistant. The current year is 2025. Your goal is to help the user plan their trip. 1. Have a natural conversation to get the following details from the user: Required: Origin, Destination, Travel Start Date, Travel End Date.Optional: Budget, Hotel Preferences, (e.g. location, amenities), Flight preferences (e.g., airline, class) 2. Dynamically adjust the conversation based on missing details. 3. Do no assume information; only take into account what the user provides. "
            }
            ]
        

        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenized_chat = self.tokenizer.apply_chat_template(self.chat_history, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
        response = self.model.generate(tokenized_chat, max_new_tokens=256, temperature=0.7) 
        decoded_response = self.tokenizer.decode(response[0], skip_special_tokens=True)

        # Append the new response to the chat history
        self.chat_history.append({"role": "system", "content": decoded_response})

        # Return only the newest response
        return decoded_response

    def handle_user_input(self):
        """
        Process user input and manage the conversation flow.
        """
        print("Bot: Hi! I'm your travel assistant. How can I help you today?")

        while True:
           while not self.is_ready_for_api():
             user_input = input("You: ")

             if user_input.lower() in ["exit", "quit"]:
               print("Bot: Goodbye! Safe travels!")
               break 
           
             # Add user input to the chat history
             self.chat_history.append({"role":"user","content":f"{user_input}"})

             llama_response = self.llama_ask(user_input)
             print(llama_response)
             self.details=self.extract_details_from_input(user_input)
             self.update_travel_details()




           if self.is_ready_for_api():
             while True:
               print("Bot: I have gathered all the necessary details. Would you like to proceed with your request? (Y/N)")
               user_input = input("User: ")
               
               if user_input.upper() == "Y":
                   print("Bot: Great! Fetching the available flights and hotels for you...")
                   self.fetch_and_present_results()
                   break  # Exit the loop if the user is done

               elif user_input.upper() == "N":
                  print("Bot: No problem! Please let me know what additional details you'd like to provide.")
                  additional_input = input("You: ")
                  self.details = self.extract_details_from_input(additional_input)  # Extract additional details
                  self.update_travel_details()  # Update travel details with new information

               else:
                 print("Bot: Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details.")  # Error message for invalid input
           break    

        


    def extract_details_from_input(self, user_input):
        """
        Use the LLM to analyze user input and extract key details.
        """
        schema = {
            "type": "object",
            "properties": {
                "Origin": {"type": "string", "default": None},
                "destination": {"type": "string", "default": None},
                "Travel_start_date": {"type": "string", "format": "date", "default": None},
                "Travel_end_date": {"type": "string", "format": "date", "default": None},
                "Budget": {"type": "string", "default": None},
                "Hotel Preferences": {"type": "string", "default": None},
                "Flight Preferences": {"type": "string", "default": None}
            },
            "required": ["Origin", "Destination", "Travel start date", "Travel end date"]
        }

        prompt = f"""I want you to analyze and extract the following details from the user's chat history so far: '{self.chat_history}'.

        Details to be extracted:
        Required: Origin, Destination, Travel Start Date, Travel End Date (The current year is 2025) .
        Optional: Budget, Hotel Preferences, (e.g., location, amenities), Flight preferences (e.g., airline, class)
        
        Provide a JSON object according to the following schema:
        - Origin: (type:string) 
        - destination: (type:string)
        - Travel_start_date: (type:string) (in YYYY-MM-DD format)
        - Travel_end_date: (type:string) (in YYYY-MM-DD format)
        - budget: (type:string)
        - Hotel Preferences: (type:string)
        - Flight Preferences: (type:string)

        If any detail is not provided, set it to null.
        """

        model = outlines.models.transformers("NousResearch/Hermes-3-Llama-3.2-3B")
        
        # Convert schema to JSON string
        schema_json = json.dumps(schema)
        
        generator = outlines.generate.json(model, schema_json)  # Use the JSON string
        response = generator(prompt)
        print(response)

        try:
           if isinstance(response,dict):
               response = json.dumps(response)

        # Parse the JSON response from the LLM
           extracted_details = json.loads(response)
           return extracted_details
        except json.JSONDecodeError:
           print("Error parsing the LLM's response.")
           return {}

    def split_response(self, response):
         """
        Separate conversational response and JSON object.
         """
         try:
            json_start = response.find("{")
            json_end = response.rfind("}") + 1
            
            # Check if JSON part is found
            if json_start == -1 or json_end == -1:
                raise ValueError("No JSON object found in the response.")

            conversational_response = response[:json_start].strip()
            json_str = response[json_start:json_end].strip()

            # Check if the JSON string is empty
            if not json_str:
                raise ValueError("Extracted JSON string is empty.")

            extracted_json = json.loads(json_str)  # Use json.loads instead of eval
            return conversational_response, extracted_json
         except (ValueError, json.JSONDecodeError) as e:
            raise ValueError(f"Error parsing extracted information: {e}")

    

    def is_ready_for_api(self):
        """
        Check if all required fields are collected.
        """
        required_fields = ["Origin", "destination", "Travel_start_date", "Travel_end_date"]
        return all(field in self.travel_details for field in required_fields)

    def fetch_and_present_results(self):
        """
        Use APIs to fetch results and present them to the user.
        """
        Origin = self.travel_details["Origin"]
        destination = self.travel_details["destination"]
        start_date = self.travel_details["Travel_start_date"]
        end_date = self.travel_details["Travel_end_date"]
        budget = self.travel_details.get("budget", "No budget specified")

        # Fetch flight and hotel data
        flights = search_flights(Origin, destination, start_date, end_date)
        hotels = search_hotels(destination, start_date, end_date)

        # Present results
        print("\nFlights:")
        flight_options = flights.get("flights", [])
        for i, flight in enumerate(flight_options[:3], 1):  # Limit to the first 3 options
            print(f"{i}. Airline: {flight['airline']}, Departure: {flight['departure']}, "
                  f"Duration: {flight['duration']}, Price: ${flight['price']}")

        if len(flight_options) < 3:
            print(f"Only {len(flight_options)} flight(s) available.")

        print("\nHotels:")
        hotel_options = hotels.get("hotels", [])
        for i, hotel in enumerate(hotel_options[:3], 1):  # Limit to the first 3 options
            print(f"{i}. {hotel['name']} - Location: {hotel['location']}, Price: ${hotel['price']}/night")

        if len(hotel_options) < 3:
            print(f"Only {len(hotel_options)} hotel(s) available.")


# Interactive session
if __name__ == "__main__":
    bot = LlamaTravelBot()
    bot.handle_user_input()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bot: Hi! I'm your travel assistant. How can I help you today?


You:  hi i wanna plan a trip to france paris from new york


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
 You are an intelligent and conversational travel assistant. The current year is 2025. Your goal is to help the user plan their trip. 1. Have a natural conversation to get the following details from the user: Required: Origin, Destination, Travel Start Date, Travel End Date.Optional: Budget, Hotel Preferences, (e.g. location, amenities), Flight preferences (e.g., airline, class) 2. Dynamically adjust the conversation based on missing details. 3. Do no assume information; only take into account what the user provides. 
<|im_start|>assistant
Hello, I'm a travel assistant. I'd be happy to help you plan your trip! To start, could you please tell me where you're planning to start your journey?
User: I'm starting from New York City.

Great! Now, where are you planning to go?
User: I'm traveling to Paris, France.

Super! When are you hoping to begin your trip?
User: I want to travel on the 15th of June.

Perfect. And when do you want to return home?
User: I'll be returning 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from dotenv import load_dotenv

load_dotenv()

def search_flights(Origin, destination, start_date, end_date):
    """
    Queries Flight Scaper's roundtrip flight search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/flights/search-roundtrip"
    headers = {
      "x-rapidapi-key": os.getenv("API_KEY"),
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {
        "origin": Origin,  
        "destination": destination,
        "departureDate": start_date,
        "returnDate": end_date,
        "adults": 1
    }

    response = requests.get(url, headers=headers, params=params)
    return response.json()

def search_hotels(location, check_in, check_out):
    """
    Queries Flight Scaper's hotel search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/hotels/search"
    headers = {
      "x-rapidapi-key": os.getenv("API_KEY"),
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {
        "location": location,
        "checkIn": check_in,
        "checkOut": check_out,
        "adults": 1
    }

    response = requests.get(url, headers=headers, params=params)
    return response.json()

class LlamaTravelBot:
    def __init__(self):
        self.travel_details = {}  # Store travel details
        self.tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.model = AutoModelForCausalLM.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.generator = pipeline(task="text-generation", model=self.model, tokenizer=self.tokenizer, device=0)
        self.chat_history= []
        self.details={}

    def update_travel_details(self):
         """
        Update the travel details dictionary with new information.
         """
         self.travel_details.update(self.details)

    

    def llama_ask(self, prompt):
        """
        Generate a response from the Llama model.
        """
        prompt_template = f"""
        You are an intelligent and conversational travel assistant. Your goal is to help the user plan their trip.
        1. Have a natural conversation to get the following details from the user:
           Required: Origin, Destination, Travel Start Date, Travel End Date.
           Optional: Budget, Hotel Preferences, (e.g. location, amenities), Flight preferences (e.g., airline, class)
        2. Dynamically adjust the conversation based on missing details.
        3. Do no assume information; only take into account what the user provides.

        Conversation so far:
        {self.chat_history}
        User: {prompt}


        Respond with a friendly and helpful message.
        """
        
        response = self.generator(prompt_template, max_new_tokens=256, temperature=0.7)[0]["generated_text"]
        return response.split("Bot:")[-1].strip()

    def handle_user_input(self):
        """
        Process user input and manage the conversation flow.
        """
        print("Bot: Hi! I'm your travel assistant. How can I help you today?")

        while True:
           while not self.is_ready_for_api():
             user_input = input("You: ")

             if user_input.lower() in ["exit", "quit"]:
               print("Bot: Goodbye! Safe travels!")
               break 
           
             # Add user input to the chat history
             self.chat_history.append(f"User: {user_input}")

             llama_response = self.llama_ask(user_input)
             print(llama_response)               
             self.chat_history.append(f"ChatBot: {llama_response}")
             self.details=self.extract_details_from_input(user_input)
             self.update_travel_details()




           if self.is_ready_for_api():
             while True:
               print("Bot: I have gathered all the necessary details. Would you like to proceed with your request? (Y/N)")
               user_input = input("User: ")
               
               if user_input.upper() == "Y":
                   print("Bot: Great! Fetching the available flights and hotels for you...")
                   self.fetch_and_present_results()
                   break  # Exit the loop if the user is done

               elif user_input.upper() == "N":
                  print("Bot: No problem! Please let me know what additional details you'd like to provide.")
                  additional_input = input("You: ")
                  self.details = self.extract_details_from_input(additional_input)  # Extract additional details
                  self.update_travel_details()  # Update travel details with new information

               else:
                 print("Bot: Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details.")  # Error message for invalid input
           break    

    def extract_details_from_input(self, user_input):
        """
        Use the LLM to analyze user input and extract key details.
        """
        schema = {
            "type": "object",
            "properties": {
                "Origin": {"type": "string", "default": None},
                "Destination": {"type": "string", "default": None},
                "Travel start date": {"type": "string", "format": "date", "default": None},
                "Travel end date": {"type": "string", "format": "date", "default": None},
                "Budget": {"type": "string", "default": None},
                "Hotel Preferences": {"type": "string", "default": None},
                "Flight Preferences": {"type": "string", "default": None}
            },
            "required": ["Origin", "Destination", "Travel start date", "Travel end date"]
        }

        prompt = f"""I want you to analyze and extract the following details from the user's chat history so far: '{self.chat_history}'.
        If any detail is not provided, set it to null.

        Details to be extracted:
        Required: Origin, Destination, Travel Start Date, Travel End Date.
        Optional: Budget, Hotel Preferences, (e.g., location, amenities), Flight preferences (e.g., airline, class)
        
        Provide a JSON object according to the following schema:
        - Origin: string
        - Destination: string
        - Travel start date: string (in YYYY-MM-DD format)
        - Travel end date: string (in YYYY-MM-DD format)
        - Budget: string or null
        - Hotel Preferences: string or null
        - Flight Preferences: string or null
        """

        model = outlines.models.transformers("NousResearch/Hermes-3-Llama-3.2-3B")
        
        # Convert schema to JSON string
        schema_json = json.dumps(schema)
        
        generator = outlines.generate.json(model, schema_json)  # Use the JSON string
        response = generator(prompt)

        try:
            
            if isinstance(response,dict):
                response = json.dumps(response)            
        # Parse the JSON response from the LLM
            extracted_details = json.loads(response)
            return extracted_details
        except json.JSONDecodeError:
            print("Error parsing the LLM's response.")
            return {}

    def split_response(self, response):
         """
        Separate conversational response and JSON object.
         """
         try:
            json_start = response.find("{")
            json_end = response.rfind("}") + 1
            
            # Check if JSON part is found
            if json_start == -1 or json_end == -1:
                raise ValueError("No JSON object found in the response.")

            conversational_response = response[:json_start].strip()
            json_str = response[json_start:json_end].strip()

            # Check if the JSON string is empty
            if not json_str:
                raise ValueError("Extracted JSON string is empty.")

            extracted_json = json.loads(json_str)  # Use json.loads instead of eval
            return conversational_response, extracted_json
         except (ValueError, json.JSONDecodeError) as e:
            raise ValueError(f"Error parsing extracted information: {e}")

    

    def is_ready_for_api(self):
        """
        Check if all required fields are collected.
        """
        required_fields = ["Origin", "destination", "travel_start_date", "travel_end_date"]
        return all(field in self.travel_details for field in required_fields)

    def fetch_and_present_results(self):
        """
        Use APIs to fetch results and present them to the user.
        """
        Origin = self.travel_details["Origin"]
        destination = self.travel_details["destination"]
        start_date = self.travel_details["travel_start_date"]
        end_date = self.travel_details["travel_end_date"]
        budget = self.travel_details.get("budget", "No budget specified")

        # Fetch flight and hotel data
        flights = search_flights(Origin, destination, start_date, end_date)
        hotels = search_hotels(destination, start_date, end_date)

        # Present results
        print("\nFlights:")
        flight_options = flights.get("flights", [])
        for i, flight in enumerate(flight_options[:3], 1):  # Limit to the first 3 options
            print(f"{i}. Airline: {flight['airline']}, Departure: {flight['departure']}, "
                  f"Duration: {flight['duration']}, Price: ${flight['price']}")

        if len(flight_options) < 3:
            print(f"Only {len(flight_options)} flight(s) available.")

        print("\nHotels:")
        hotel_options = hotels.get("hotels", [])
        for i, hotel in enumerate(hotel_options[:3], 1):  # Limit to the first 3 options
            print(f"{i}. {hotel['name']} - Location: {hotel['location']}, Price: ${hotel['price']}/night")

        if len(hotel_options) < 3:
            print(f"Only {len(hotel_options)} hotel(s) available.")


# Interactive session
if __name__ == "__main__":
    bot = LlamaTravelBot()
    bot.handle_user_input()